In [1]:
import pandas as pd

In [7]:
df = pd.read_csv("../data/raw/fbref_Premier-League_2023-2024_fixtures.csv")
df

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,Match Report,NaN
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,Match Report,NaN
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,Match Report,NaN
3,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,Match Report,NaN
4,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,Match Report,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,Sun,2024-05-19,16:00,Brighton,1.7,0–2,1.3,Manchester Utd,31662.0,The American Express Community Stadium,Craig Pawson,Match Report,NaN
376,38.0,Sun,2024-05-19,16:00,Brentford,1.1,2–4,3.4,Newcastle Utd,17124.0,Gtech Community Stadium,Simon Hooper,Match Report,NaN
377,38.0,Sun,2024-05-19,16:00,Manchester City,1.9,3–1,0.4,West Ham,55097.0,Etihad Stadium,John Brooks,Match Report,NaN
378,38.0,Sun,2024-05-19,16:00,Arsenal,2.9,2–1,0.6,Everton,60312.0,Emirates Stadium,Michael Oliver,Match Report,NaN


In [5]:
df.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,Match Report,NaN
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,Match Report,NaN
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,Match Report,NaN
3,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,Match Report,NaN
4,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,Match Report,NaN


In [6]:
df.isnull().sum()

Wk                0
Day               0
Date              0
Time              0
Home              0
xG                0
Score             0
xG.1              0
Away              0
Attendance        0
Venue             0
Referee           0
Match Report      0
Notes           380
dtype: int64

The notes columns is't important

In [17]:
df.drop('Notes', axis=1, inplace=True)

In [18]:
df.columns

Index(['Wk', 'Day', 'Date', 'Time', 'Home', 'xG', 'Score', 'xG.1', 'Away',
       'Attendance', 'Venue', 'Referee', 'Match Report', 'HomeGoals',
       'AwayGoals'],
      dtype='object')

In [19]:
df.isnull().sum()

Wk              0
Day             0
Date            0
Time            0
Home            0
xG              0
Score           0
xG.1            0
Away            0
Attendance      0
Venue           0
Referee         0
Match Report    0
HomeGoals       0
AwayGoals       0
dtype: int64

## Restructuring for modeling basing of the Team

In [22]:
def restructure_modeling(df):
    home = df[['Date', 'Home', 'Away', 'HomeGoals', 'AwayGoals']].copy()
    away = df[['Date', 'Away', 'Home', 'AwayGoals', 'HomeGoals']].copy()

    home.columns = ['Date', 'Team', 'Opponent', 'GoalsFor', 'GoalsAgainst']
    away.columns = ['Date', 'Team', 'Opponent', 'GoalsFor', 'GoalsAgainst']

    full = pd.concat([home, away])
    full = full.sort_values(by='Date').reset_index(drop=True)

    full['Result'] = full.apply(lambda row: 'W' if row.GoalsFor > row.GoalsAgainst else 'L' if row.GoalsFor < row.GoalsAgainst else 'D', axis=1)

    return full

In [26]:
new_df = restructure_modeling(df)
new_df

,Date,Team,Opponent,GoalsFor,GoalsAgainst,Result
0,2023-08-11,Burnley,Manchester City,0.0,3.0,L
1,2023-08-11,Manchester City,Burnley,3.0,0.0,W
2,2023-08-12,Everton,Fulham,0.0,1.0,L
3,2023-08-12,Arsenal,Nott'ham Forest,2.0,1.0,W
4,2023-08-12,Nott'ham Forest,Arsenal,1.0,2.0,L
...,...,...,...,...,...,...
755,2024-05-19,Chelsea,Bournemouth,2.0,1.0,W
756,2024-05-19,Luton Town,Fulham,2.0,4.0,L
757,2024-05-19,Liverpool,Wolves,2.0,0.0,W
758,2024-05-19,Everton,Arsenal,1.0,2.0,L


In [16]:
if 'Score' in df.columns:
        df[['HomeGoals', 'AwayGoals']] = df['Score'].str.split('–', expand=True).astype(float)
df

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,HomeGoals,AwayGoals
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,Match Report,NaN,0.0,3.0
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,Match Report,NaN,2.0,1.0
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,Match Report,NaN,0.0,1.0
3,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,Match Report,NaN,4.0,1.0
4,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,Match Report,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,Sun,2024-05-19,16:00,Brighton,1.7,0–2,1.3,Manchester Utd,31662.0,The American Express Community Stadium,Craig Pawson,Match Report,NaN,0.0,2.0
376,38.0,Sun,2024-05-19,16:00,Brentford,1.1,2–4,3.4,Newcastle Utd,17124.0,Gtech Community Stadium,Simon Hooper,Match Report,NaN,2.0,4.0
377,38.0,Sun,2024-05-19,16:00,Manchester City,1.9,3–1,0.4,West Ham,55097.0,Etihad Stadium,John Brooks,Match Report,NaN,3.0,1.0
378,38.0,Sun,2024-05-19,16:00,Arsenal,2.9,2–1,0.6,Everton,60312.0,Emirates Stadium,Michael Oliver,Match Report,NaN,2.0,1.0


In [31]:
manCity = new_df[(new_df['Team'] == 'Manchester City')]
manCity.reset_index(drop=True, inplace=True)

In [32]:
manCity

,Date,Team,Opponent,GoalsFor,GoalsAgainst,Result
0,2023-08-11,Manchester City,Burnley,3.0,0.0,W
1,2023-08-19,Manchester City,Newcastle Utd,1.0,0.0,W
2,2023-08-27,Manchester City,Sheffield Utd,2.0,1.0,W
3,2023-09-02,Manchester City,Fulham,5.0,1.0,W
4,2023-09-16,Manchester City,West Ham,3.0,1.0,W
5,2023-09-23,Manchester City,Nott'ham Forest,2.0,0.0,W
6,2023-09-30,Manchester City,Wolves,1.0,2.0,L
7,2023-10-08,Manchester City,Arsenal,0.0,1.0,L
8,2023-10-21,Manchester City,Brighton,2.0,1.0,W
9,2023-10-29,Manchester City,Manchester Utd,3.0,0.0,W
